# 순환 신경망으로 시퀀스 데이터 모델링

## Index

 - **1. 시퀀스 데이터란?**
<br><br>
 - **2. 시퀀스 모델링을 위한 RNN**
<br><br>
 - **3. Tensorflow의 tf.keras를 API로 시퀀스 모델링을 위한 다층 RNN 구현**
<br><br>
 - **4. Project1 다층 RNN으로 IMDb 영화 리뷰의 감성 분석 수행**
<br><br>
 - **5. Project2 Tensorflow로 글자 단위 언어 모델 구현**
<br><br>
 - **6. 전체 요약**

## 1. 시퀀스 데이터란?

### 1.1 시퀀스 데이터 모델링 : 순서를 고려한다.

다른 데이터 타입과 다르게 **시퀀스(sequence)**는 특별하다. 시퀀스 원소들은 특정 순서를 가지므로 상호 독립적이지 않기 때문이다.
<br><br>
일반적으로 지도 학습의 머신 러닝 알고리즘은 입력 데이터가 **독립동일분포(Independent and Identically Distributed, IDD)**라고 가정한다. 예를 들어 $n$개의 데이터 샘플 $x^{(1)}$,$x^{(2)}$,$\cdot\cdot\cdot$,$x^{(n)}$이 있을 때 머신 러닝 알고리즘을 훈련하기 위해 데이터를 사용하는 순서는 상관이 없다.
<br><br>
시퀀스 데이터를 다룰 때는 더 이상 이런 가정이 유효하지 않다. 시퀀스는 정의 잔체가 순서를 고려한 데이터이기 때문이다.

### 1.2 시퀀스 표현

입력 데이터에서 의미 있는 순서를 가지도록 시퀀스를 구성한다. 그 다음 머신 러닝 모델이 이런 유용한 정보를 사용하도록 만들어야 한다.
<br><br>
이제 시퀀스를 ($x^{(1)}$,$x^{(2)}$,$\cdot\cdot\cdot$,$x^{(T)}$)처럼 나타내자. 위의 첨자는 샘플 순서를 나타낸다. $T$는 시퀀스 길이이다. 시퀀스의 좋은 예는 **시계열 데이터(Time Series)**이다. 여기서 각 샘플 포인트 $x^{(t)}$는 특정 시간 $t$에 속한다.

다음은 시계열 데이터의 예이다. $x$와 $y$는 시간축을 따라 순서대로 나열되어 있다. 따라서 $x$와 $y$는 시퀀스 데이터이다.

![figure](./figure/figure01.png)

MLP와 CNN 같이 기본적인 신경망 모델은 입력 샘플의 순서를 다루지 못한다. 쉽게 생각해서 이런 모델들은 이전에 본 샘플을 기억핮 ㅣ못한다. 샘플이 정방향과 역방향단게를 통과하고 가중치는 샘플이 처리되는 순서와 상관없이 업데이트 된다.
<br><br>
반면에 RNN은 시퀀스 모델링을 위해 고안되었다. 지난 정보를 기억하고 이를 기반으로 새로운 이벤트를 처리할 수 있다.

### 1.3 시퀀스 모델링의 종류

시퀀스 모델링은 언어번역(영어에서 독일어로 번역), 이미지 캡셔닝(captioning), 텍스트 생성처럼 흥미로운 어플리케이션이 많이 있다.
<br><br>
적절한 모델을 개발하기 위해 시퀀스 모델링의 종류를 이해할 필요가 있다. 다음 그림은 각기 다른 종류의 입력과 출력 데이터에 대한 관계를 보여준다.

![figure](./figure/figure02.jpeg)

어떤 입력 데이터와 출력 데이터가 있다고 가정해보자. 입력과 출력 데이터가 시퀀스로 표현되지 않는다면 일반 데이터로 처리한다. 이런 데이터를 모델링하려면 MLP나 CNN 같은 방법 중 하나를 사용할 수 있다. 입력이나 출력이 시퀀스라면 다음 3가지 중 하나로 구성된다.


 - **many-to-one** : 입력 데이터가 시퀀스이다. 출력은 시퀀스가 아니라 고정 크기의 벡터이다. 예를 들어 감성 분석에서 입력은 텍스트이고 출력은 클래스 레이블인 경우이다.
 - **one-to-many** : 입력 데이터가 시퀀스가 아니라 일반적이 형태이다. 출력은 시퀀스이다. 이런 종류의 예로는 이미지 캡셔닝이 있다. 입력이 이미지이고 출력은 영어 문장이다.
 - **many-to-oㅡmany** : 입력과 출력이 모두 시퀀스이다. 이런 종류는 입력과 출력이 동기적인지 아닌지에 따라 더 나눌 수 있다. 동기적인 many-to-many 모델링 작업의 예로는 각 프레임이 레이블 되어 있는 비디오 분류이다. 그렇지 않은 many-to-many 모델의 예는 한 언어에서 다른 언어로 번역하는 작업이다. 예를 들어 독일어로 번역하기 전에 전체 영어 문자을 읽어 처리한다. 

## 2. 시퀀스 모델링을 위한 RNN

### 2.1 RNN 구조와 데이터 흐름 이해

RNN 구조를 이해하기 위해 기본 피드포워드 신경망과 RNN을 나란히 살펴보자.

![figure](./figure/figure03.png)

두 네트워크 모두 하나의 은닉층만이 있다. 가ㅣ본 피드포워드 네트워크에서 정보는 입력에서 은닉층으로 흔 후 은닉층에서 출력층으로 전달된다. 하지만 RNN은 은닉층이 입력층과 이전 타임 스텝(time step)의 은닉층으로부터 정보를 받는다.
<br><br>
인접한 타임스텝의 정보가 은닉층에 흐르기 때문에 네트워크가 이전 이벤트를 기억할 수 있다. 이 정보 흐름은 보통 루프(loop)로 표시한다. 그래프 표기법에서는 순환 에지(recurrent edge)라고도 하기 때문에 이구조 이름이 여기서 유래되었다.

다음 그림은 단이층 RNN과 다층 RNN을 비교하여 보여준다.